In [1]:
import pandas as pd
import numpy as np 
import time

score_df = pd.DataFrame([[1,50,80,70,'boy',1], 
              [2,60,45,50,'boy',2],
              [3,98,43,55,'boy',1],
              [4,70,69,89,'boy',2],
              [5,56,79,60,'girl',1],
              [6,60,68,55,'girl',2],
              [7,45,70,77,'girl',1],
              [8,55,77,76,'girl',2],
              [9,25,57,60,'girl',1],
              [10,88,40,43,'girl',3],
              [11,25,60,45,'boy',3],
              [12,80,60,23,'boy',3],
              [13,20,90,66,'girl',3],
              [14,50,50,50,'girl',3],
              [15,89,67,77,'girl',3]],columns=['student_id','math_score','english_score','chinese_score','sex','class'])
score_df


# 在速度較慢的時候，可以從哪裡開始檢查?
# 資料過大時應採取什麼方式讓記憶體占用量下降？

,student_id,math_score,english_score,chinese_score,sex,class
0,1,50,80,70,boy,1
1,2,60,45,50,boy,2
2,3,98,43,55,boy,1
3,4,70,69,89,boy,2
4,5,56,79,60,girl,1
5,6,60,68,55,girl,2
6,7,45,70,77,girl,1
7,8,55,77,76,girl,2
8,9,25,57,60,girl,1
9,10,88,40,43,girl,3


In [ ]:
# Question : 在速度較慢的時候，可以從哪裡開始檢查?

In [6]:
# 在速度較慢的時候，可以從哪裡開始檢查?
# 1. agg使用Python的內建函式
star_time = time.time()
score_df.groupby('class').agg('mean')
end_time = time.time()
end_time - star_time


0.002010822296142578

In [7]:
# 2. agg使用自定義函式
star_time = time.time()
score_df.groupby('class').agg(lambda x: x.mean())
end_time = time.time()
end_time - star_time

0.007999897003173828

In [8]:
# 3. transform使用Python的內建函式
star_time = time.time()
score_df.groupby('class').transform('mean')
end_time = time.time()
end_time - star_time

0.011001110076904297

In [9]:
# 4. transform使用自定義函式
star_time = time.time()
score_df.groupby('class').transform(lambda x: x.mean())
end_time = time.time()
end_time - star_time

0.0070073604583740234

In [ ]:
# Answer: 從以上4個方法中可以看出使用agg+python內建函式最快，使用agg+自訂函式所花時間最長。

In [ ]:
# Question : 資料過大時應採取什麼方式讓記憶體占用量下降？

In [10]:
#遇到大資料集時，常有記憶體不足的問題。為改善部分不同所以分成浮點數float與整數int的資料集，可以看到不管浮點數還是整數都佔了800128bytes。
float_data = pd.DataFrame(np.random.uniform(0,5,100000).reshape(1000,100))
int_data = pd.DataFrame(np.random.randint(0,1000,100000).reshape(1000,100))
int_data.memory_usage(deep=True).sum(), float_data.memory_usage(deep=True).sum()

(400128, 800128)

In [11]:
# 1. 整數型態int改成uint減少記憶體正用空間，使用前800128bytes，使用後剩下200128bytes
downcast_int = int_data.apply(pd.to_numeric,downcast='unsigned')
int_data.memory_usage(deep=True).sum(),downcast_int.memory_usage(deep=True).sum()

(400128, 200128)

In [12]:
# 2. 原本有100個欄位是int64，經過downcast變成了100個欄位的uint16
compare_int = pd.concat([int_data.dtypes,downcast_int.dtypes],axis=1)
compare_int.columns = ['before','after']
compare_int.apply(pd.value_counts)

,before,after
uint16,NaN,100.0
int32,100.0,NaN


In [13]:
# 3. 浮點數型態float64改成float32減少記憶體正用空間，使用前800128bytes，使用後剩下400128bytes
downcast_float = float_data.apply(pd.to_numeric,downcast='float')
float_data.memory_usage(deep=True).sum(),downcast_float.memory_usage(deep=True).sum()

(800128, 400128)

In [14]:
# 4. 原本有100個欄位是float64，經過downcast變成了100個欄位的float32
compare_int = pd.concat([float_data.dtypes,downcast_float.dtypes],axis=1)
compare_int.columns = ['before','after']
compare_int.apply(pd.value_counts)

,before,after
float32,NaN,100.0
float64,100.0,NaN


In [ ]:
# Answer:   1. 從以上4個方法中可以看出使用float的資料型態 > uint > int的資料型態。
#           2. 使用downcast可以減少使用的記憶體用量